# AI/ML methods notebook



# install Python packages
This notebook is equipped with a dedicated login shell, tailored to the environment in which it is executed. If you are utilizing your personal compute system, such as a laptop, the login corresponds to your individual compute system login. Conversely, when running this notebook on Google Colab, the login is attributed to the root user. The initiation of Linux shell commands within Jupyter notebook code cells is denoted by a preceding exclamation point (!).

In the code cell below, the provided pip commands are employed to install a range of Python libraries essential for the tasks covered in this notebook. It's worth noting that additional Python libraries are automatically installed within our virtual environment.

In [ ]:
!pip install scikit-learn --no-cache
!pip install scanpy --no-cache
!pip install gseapy --no-cache
!pip install pybiomart==0.1 --no-cache
#!pip install mygene --no-cache
!pip install sklearn_som  --no-cache
!pip install pandas --no-cache
!pip install numpy --no-cache
!pip install matplotlib --no-cache
!pip install sklearn-som --no-cache
!pip install pyDeseq2 --no-cache
!pip install Ensembl_converter --no-cache
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 250.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 220.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 191.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.8/590.8 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 201.9 MB/s eta 0:00:00
  Created wheel for pybiomart: filename=pybiomart-0.1-py3-none-any.whl size=14600 sha256=d4d31738c1dcfb41fb1eeb810ffb7aac2beba02affe28be2b46635baf8419e13
  Stored in directory: /tmp/pip-ephem-wheel-cache-9s8czuel/wheels/ad/45

# import Python modules

This notebook imports a number of Python modules for use in several notebooks.

In [ ]:
import re
import requests
import json
import pandas as pd
from urllib.request import urlretrieve
import numpy as np
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import scanpy as sc
import gseapy as gp
from gseapy.plot import gseaplot
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from gseapy import Msigdb
from pybiomart import Server
#import mygene
import seaborn as sns
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import TweedieRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from math import log
import statsmodels.api as sm
import pylab
import operator
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, PrecisionRecallDisplay
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from itertools import islice
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import argparse
import scipy.stats as stats
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')
from Ensembl_converter import EnsemblConverter
import shap
from sklearn.inspection import permutation_importance
from google.colab import data_table
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
#from vega_datasets import data

# define misc helper methods

In [ ]:
def set_maxdisplay(n=None):
  pd.set_option('display.max_rows', n)
  from notebook.services.config import ConfigManager
  cm = ConfigManager().update('notebook', {'limit_output': n})

# Define data ingestion methods

In [ ]:
def read_meta_data(dataset):
  # dataset=255
  url = 'https://osdr.nasa.gov/geode-py/ws/studies/OSD-' + str(dataset) + '/download?source=datamanager&file=OSD-' + dataset + '_metadata_OSD-' + dataset + '-ISA.zip'
  filename = dataset + '-meta.zip'
  urlretrieve(url, filename)
  !unzip -o {filename} > /dev/null
  df = pd.read_csv('s_OSD-' + dataset + '.txt', sep='\t', header=0)
  return df

In [ ]:
def read_behavior_data(dataset, data):
  # dataset = '557'
  # data = 'LSDS-1_immunostaining_microscopy_PNAtr_Transformed_Reusable_Results'
  url='https://osdr.nasa.gov//geode-py/ws/studies/OSD-' + str(dataset) + '/download?file=' + data + '.csv&version=1'
  df = pd.read_csv(url)
  return df

In [ ]:
def read_flowcytometry_data(dataset, data):
  # dataset = '557'
  # data = 'LSDS-1_immunostaining_microscopy_PNAtr_Transformed_Reusable_Results'
  url='https://osdr.nasa.gov//geode-py/ws/studies/OSD-' + str(dataset) + '/download?file=' + data + '.csv&version=1'
  df = pd.read_csv(url)
  return df

In [ ]:
def generate_and_display_confusion_matrix(Y_test, final_predictions):

  # Convert string labels to numerical labels 0 and 1
  Y_test_numeric = np.where(Y_test == 'pos', 1, 0)
  final_predictions_numeric = np.where(final_predictions == 'pos', 1, 0)

  # Confusion Matrix
  cm = confusion_matrix(Y_test_numeric, final_predictions_numeric)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["neg", "pos"])
  print("Confusion Matrix:\n", cm)

  # Plot
  disp.plot(cmap='Blues')
  plt.show()

# define data filtering methods

In [ ]:
def extract_digit(sample_name):
    try:
        # Convert to string, split, and get the second element (index 1)
        digit = str(sample_name).split('_')[0]
        # Attempt to convert to an integer, if it fails return None
        return int(digit)
    except (IndexError, ValueError, TypeError):
        return None

In [ ]:
def extract_assay(sample_name):
    try:
        parts = str(sample_name).split('_')
        assay_name = parts[1]
        return assay_name
    except (IndexError, TypeError):
        return None

In [ ]:
def drop_nans(df):
  # drop NaN rows
  df.dropna(inplace=False)
  return df


# data transformation methods

In [ ]:
def prepare_data_for_ml(x, y, test_size=0.2, random_state=42):

    # Split data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)

    # Separate numerical and categorical columns
    df_num = x.drop(['Factor Value[Sex]', 'Factor Value[Treatment]', 'Factor Value[Ionizing Radiation]'], axis=1)
    df_cat_treatment = x[['Factor Value[Treatment]']]
    df_cat_gender = x[['Factor Value[Sex]']]
    df_cat_rad = x[['Factor Value[Ionizing Radiation]']]

    # Numerical pipeline
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", StandardScaler())
    ])

    # Categorical pipelines (One-hot encoding)
    categorical_features = ['Factor Value[Sex]', 'Factor Value[Treatment]', 'Factor Value[Ionizing Radiation]']
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_pipeline, list(df_num.columns)),
            ('cat', categorical_transformer, categorical_features)])

    # Fit and transform the data
    X_train_prepared = preprocessor.fit_transform(X_train)
    X_test_prepared = preprocessor.transform(X_test)

    # Get column names for transformed data
    num_columns = df_num.columns
    cat_columns = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
    all_columns = np.concatenate([cat_columns, num_columns])

    # Convert transformed data to DataFrames
    X_train_prepared_df = pd.DataFrame(X_train_prepared, columns=all_columns, index=X_train.index)
    X_test_prepared_df = pd.DataFrame(X_test_prepared, columns=all_columns, index=X_test.index)

    return X_train_prepared_df, X_test_prepared_df, Y_train, Y_test

# plotting methods

In [ ]:
def visualize_numerical_features(dataframe, cols_per_row=3, bins=10):

    dataframe['RAWM_D1_Block1_Errors'] = pd.to_numeric(dataframe['RAWM_D1_Block1_Errors'], errors='coerce')
    numerical_cols = dataframe.select_dtypes(include=['number']).columns
    numerical_cols = numerical_cols[numerical_cols != 'Source Name']

    num_plots = len(numerical_cols)
    num_rows = (num_plots + cols_per_row - 1) // cols_per_row

    fig, axes = plt.subplots(num_rows, cols_per_row, figsize=(17, 5 * num_rows))
    axes = axes.flatten()

    for i, col in enumerate(numerical_cols):
        if i < num_plots:
            axes[i].hist(dataframe[col], bins=bins)
            axes[i].set_title(col)
            axes[i].set_xlabel(col)
            axes[i].set_ylabel("Frequency")
        else:
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_target_distribution(dataframe, target_column):
    # Calculate class counts and distribution
    class_counts = dataframe[target_column].value_counts()
    class_distribution = class_counts / len(dataframe)

    # Print class counts and distribution
    print("Class Counts:\n", class_counts)
    print("\nClass Distribution:\n", class_distribution)

    # Create a bar plot
    plt.figure(figsize=(8, 6))  # Adjust figure size if needed
    class_distribution.plot(kind='bar', color=['skyblue', 'salmon'])  # Customize colors
    plt.title(f'Class Distribution of {target_column}')
    plt.xlabel('Class')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)  # Keep x-axis labels horizontal
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

In [ ]:
def plot_roc_curve(model, X_test, Y_test, pos_label='pos'):

  plt.figure(figsize=(10, 8))

  # Get predicted probabilities for the positive class
  clf_probs = model.predict_proba(X_test)[:, 1]

  # Calculate ROC curve and AUC
  fpr, tpr, thresholds = roc_curve(Y_test, clf_probs, pos_label=pos_label)
  roc_auc = auc(fpr, tpr)

  # Plot the ROC curve
  plt.plot(fpr, tpr, lw=2, label=f'{model.__class__.__name__} (AUC = {roc_auc:.2f})')

  # Plot the diagonal line (random classifier)
  plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')

  # Set plot limits and labels
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend(loc="lower right")
  plt.show()

# machine learning methods


In [ ]:
def train_and_evaluate_models(X_train_prepared_df, X_test_prepared_df, Y_train, Y_test, models_to_use = None):
  models={
  'LogisticRegression':LogisticRegression(),
  'KNeighborsClassifier':KNeighborsClassifier(),
  'SVC':SVC(probability=True),
  'DecisionTreeClassifier':DecisionTreeClassifier(),
  'RandomForestClassifier':RandomForestClassifier(),
  }

  if models_to_use:
    # Filter models dictionary based on models_to_use list
    models = {key: value for key, value in models.items() if key in models_to_use}

  for model_name, model in models.items():
    #training the model on training data
    model.fit(X_train_prepared_df,Y_train)
    #making predictions on the test data
    Y_pred=model.predict(X_test_prepared_df)
    #printing classification report which contains accuracy
    print(models[model_name], classification_report(Y_test, Y_pred))
  return models

In [ ]:
def find_best_knn_model(X_train, y_train):
  param_grid = [
      {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
  ]

  knn_clf = KNeighborsClassifier()
  grid_search = GridSearchCV(knn_clf, param_grid, cv=5,
                             scoring='accuracy', return_train_score=True)
  grid_search.fit(X_train, y_train)

  print("Best parameters:", grid_search.best_params_)
  print("Best cross-validation score:", grid_search.best_score_)

  return grid_search.best_estimator_